In [1]:
import numpy as np
import pandas as pd 
import os

# Import required modules

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Input,Model
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

# Input Data and Data Argumentation 

In [3]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                shear_range=0.2,
                                zoom_range=0.2,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                fill_mode='nearest')

valid_gen=ImageDataGenerator(rescale=1./255)

batch_size=32
base_dir=r"C:\Users\psath\OneDrive\Desktop\pd\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)"

training_set=train_datagen.flow_from_directory(base_dir+'\\train',
                                              target_size=(224,224),
                                              batch_size=batch_size,
                                              class_mode='categorical')

valid_set=valid_gen.flow_from_directory(base_dir+'\\valid',
                                       target_size=(224,224),
                                       batch_size=batch_size,
                                       class_mode='categorical')

Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.


In [4]:
train_num=training_set.samples
valid_num=valid_set.samples

# Transfer Learning with VGG

In [5]:
base_model=tf.keras.applications.VGG16(include_top=False,
                                      weights='imagenet',
                                      input_shape=(224,224,3))
base_model.summary()

58892288/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________________________

In [6]:
base_model.trainable=False

In [7]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [8]:
inputs=Input(shape=(224,224,3))
x=base_model(inputs,training=False)
x=GlobalAveragePooling2D()(x)
x=Dense(512,activation='relu')(x)
x=Dropout(0.2)(x)
x=Dense(512,activation='relu')(x)
x=Dropout(0.2)(x)
outputs=Dense(38,activation='softmax')(x)


vgg_model=Model(inputs,outputs)

vgg_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0     

In [9]:
vgg_model.compile(optimizer=tf.keras.optimizers.Adam(),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [10]:
len(vgg_model.trainable_variables)

6

In [11]:
vgg_model.evaluate(valid_set)

550/550 [==============================] - 72s 130ms/step - loss: 3.6983 - accuracy: 0.0186


[3.6983144283294678, 0.018609151244163513]

# Train the model

In [12]:
weightpath='best_weights_9.hdf5'
checkpoint = ModelCheckpoint(weightpath, monitor='val_acc', verbose=1, 
                             save_best_only=True, save_weights_only=True, mode='max')

history=vgg_model.fit_generator(training_set,
                        steps_per_epoch=150,
                        epochs=15,
                        validation_data=valid_set,
                        validation_steps=100,
                        callbacks=[checkpoint])

Epoch 1/15
150/150 [==============================] - 79s 528ms/step - loss: 2.9487 - accuracy: 0.2052 - val_loss: 1.8998 - val_accuracy: 0.4197
Epoch 2/15
150/150 [==============================] - 78s 520ms/step - loss: 1.6170 - accuracy: 0.5031 - val_loss: 1.1576 - val_accuracy: 0.6356
Epoch 3/15
150/150 [==============================] - 77s 513ms/step - loss: 1.2345 - accuracy: 0.6106 - val_loss: 0.9528 - val_accuracy: 0.6856
Epoch 4/15
150/150 [==============================] - 77s 516ms/step - loss: 1.0450 - accuracy: 0.6587 - val_loss: 0.7467 - val_accuracy: 0.7616
Epoch 5/15
150/150 [==============================] - 78s 517ms/step - loss: 0.9384 - accuracy: 0.6988 - val_loss: 0.7878 - val_accuracy: 0.7391
Epoch 6/15
150/150 [==============================] - 76s 509ms/step - loss: 0.8428 - accuracy: 0.7256 - val_loss: 0.7110 - val_accuracy: 0.7634
Epoch 7/15
150/150 [==============================] - 76s 504ms/step - loss: 0.8419 - accuracy: 0.7248 - val_loss: 0.6394 - val_ac

In [13]:
history=vgg_model.fit_generator(training_set,
                        steps_per_epoch=150,
                        epochs=15,
                        validation_data=valid_set,
                        validation_steps=100,
                        callbacks=[checkpoint])

Epoch 1/15
150/150 [==============================] - 72s 480ms/step - loss: 0.6118 - accuracy: 0.7992 - val_loss: 0.4399 - val_accuracy: 0.8575
Epoch 2/15
150/150 [==============================] - 72s 483ms/step - loss: 0.6101 - accuracy: 0.8006 - val_loss: 0.4882 - val_accuracy: 0.8319
Epoch 3/15
150/150 [==============================] - 73s 488ms/step - loss: 0.5729 - accuracy: 0.8079 - val_loss: 0.4666 - val_accuracy: 0.8422
Epoch 4/15
150/150 [==============================] - 73s 486ms/step - loss: 0.5731 - accuracy: 0.8027 - val_loss: 0.4407 - val_accuracy: 0.8475
Epoch 5/15
150/150 [==============================] - 72s 479ms/step - loss: 0.5456 - accuracy: 0.8192 - val_loss: 0.5211 - val_accuracy: 0.8284
Epoch 6/15
150/150 [==============================] - 72s 482ms/step - loss: 0.5158 - accuracy: 0.8279 - val_loss: 0.4536 - val_accuracy: 0.8487
Epoch 7/15
150/150 [==============================] - 72s 483ms/step - loss: 0.5476 - accuracy: 0.8198 - val_loss: 0.4464 - val_ac

In [14]:
history=vgg_model.fit_generator(training_set,
                        steps_per_epoch=150,
                        epochs=15,
                        validation_data=valid_set,
                        validation_steps=100,
                        callbacks=[checkpoint])

Epoch 1/15
150/150 [==============================] - 72s 478ms/step - loss: 0.4846 - accuracy: 0.8394 - val_loss: 0.3568 - val_accuracy: 0.8803
Epoch 2/15
150/150 [==============================] - 71s 472ms/step - loss: 0.4726 - accuracy: 0.8471 - val_loss: 0.4167 - val_accuracy: 0.8600
Epoch 3/15
150/150 [==============================] - 72s 481ms/step - loss: 0.4606 - accuracy: 0.8456 - val_loss: 0.3236 - val_accuracy: 0.8844
Epoch 4/15
150/150 [==============================] - 73s 484ms/step - loss: 0.4580 - accuracy: 0.8419 - val_loss: 0.3499 - val_accuracy: 0.8791
Epoch 5/15
150/150 [==============================] - 73s 485ms/step - loss: 0.4269 - accuracy: 0.8537 - val_loss: 0.3286 - val_accuracy: 0.8900
Epoch 6/15
150/150 [==============================] - 72s 483ms/step - loss: 0.4621 - accuracy: 0.8445 - val_loss: 0.3149 - val_accuracy: 0.8966
Epoch 7/15
150/150 [==============================] - 72s 477ms/step - loss: 0.4412 - accuracy: 0.8521 - val_loss: 0.3629 - val_ac

# Fine Tuning

In [15]:
base_model.trainable=True 
vgg_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0     

In [17]:
vgg_model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

fine_tune_history=vgg_model.fit_generator(training_set,
                                        steps_per_epoch=150,
                                        validation_data=valid_set,
                                        epochs=10,
                                        validation_steps=100)

Epoch 1/10
150/150 [==============================] - 81s 541ms/step - loss: 0.4236 - accuracy: 0.8627 - val_loss: 0.3258 - val_accuracy: 0.8891
Epoch 2/10
150/150 [==============================] - 81s 539ms/step - loss: 0.3407 - accuracy: 0.8838 - val_loss: 0.1751 - val_accuracy: 0.9394
Epoch 3/10
150/150 [==============================] - 80s 536ms/step - loss: 0.2946 - accuracy: 0.9019 - val_loss: 0.2235 - val_accuracy: 0.9191
Epoch 4/10
150/150 [==============================] - 80s 536ms/step - loss: 0.2565 - accuracy: 0.9075 - val_loss: 0.1980 - val_accuracy: 0.9309
Epoch 5/10
150/150 [==============================] - 81s 541ms/step - loss: 0.2391 - accuracy: 0.9213 - val_loss: 0.2124 - val_accuracy: 0.9306
Epoch 6/10
150/150 [==============================] - 81s 543ms/step - loss: 0.1980 - accuracy: 0.9337 - val_loss: 0.2316 - val_accuracy: 0.9187
Epoch 7/10
150/150 [==============================] - 81s 538ms/step - loss: 0.2173 - accuracy: 0.9254 - val_loss: 0.1623 - val_ac